# **Data Collection - Fetch Dataset from Kaggle**

## Objectives

* Write here your notebook objective, for example, "Fetch data from Kaggle and save as raw data", or "engineer features for modelling"
* Fetch data from Kaggle and save as raw data to prepare it for the following processes.

## Inputs

* Write here which data or information you need to run the notebook
* Kaggle JSON file (authentication tokem) 

## Outputs

* Write here which files, code or artefacts you generate by the end of the notebook
* Generate Dataset: inputs/datasets/cherry_leaves 

## Additional Comments

* In case you have any additional comments that don't fit in the previous bullets, please state them here. 
* No comments



## Import pagkages

In [1]:
! pip install -r /workspaces/mildew-detection-in-cherry-leaves/requirements.txt

0/1.0 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.0/39.0 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 20.9 MB/s eta 0:00:00
 

In [2]:
import numpy
import os

---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

* We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [3]:
import os
current_dir = os.getcwd()
current_dir

'/workspaces/mildew-detection-in-cherry-leaves/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [4]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [5]:
current_dir = os.getcwd()
current_dir

'/workspaces/mildew-detection-in-cherry-leaves'

# Install Kaggle

In [6]:
! pip install kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 16.8 MB/s eta 0:00:00
  Created wheel for kaggle: filename=kaggle-1.5.13-py3-none-any.whl size=77717 sha256=1ee9b062a57c1c2e95eafb6d13ceadf50433bd66ac740d6ac71431bbed6285e7
  Stored in directory: /home/codeany/.cache/pip/wheels/e6/8e/67/e07554a720a493dc6b39b30488590ba92ed45448ad0134d253
Successfully built kaggle


---

Run the cell below to change the kaggle configuration directory to current working directory and permission of kaggle authentication json

In [8]:
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

* Get the dataset path from Kaggle url.  When you are viewing the dataset at Kaggle, check what is after https://www.kaggle.com/ (in some case kaggle.com/datasets). You should copy that at KaggleDatasetPath.

* Set the destination folder
* Set the Kaggle Dataset and Download it

In [15]:
KaggleDatasetPath = "codeinstitute/cherry-leaves"
DestinationFolder = "inputs/cherry-leaves_dataset"   
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}

 96%|█████████████████████████████████████▌ | 53.0M/55.0M [00:00<00:00, 190MB/s]
100%|███████████████████████████████████████| 55.0M/55.0M [00:00<00:00, 181MB/s]


* Un-zip the downloaded file and delete the zip file afterwards.

In [16]:
import zipfile
with zipfile.ZipFile(DestinationFolder + '/cherry-leaves.zip', 'r') as zip_ref:
    zip_ref.extractall(DestinationFolder)

os.remove(DestinationFolder + '/cherry-leaves.zip')

# Data Preparation

* Check and remove non-image files

In [18]:
def remove_non_image_file(my_data_dir):
    image_extension = ('.png', '.jpg', '.jpeg')
    folders = os.listdir(my_data_dir) 
    for folder in folders:
        files = os.listdir(my_data_dir + '/' + folder)
        
            #print(files)
        i = []
        j = []
        for given_file in files:
            if not given_file.lower().endswith(image_extension):
                file_location = my_data_dir + '/' + folder + '/' + given_file
                os.remove(file_location) # remove non image file
                i.append(1)
            else:
                j.append(1)
                pass
        print(f"Folder: {folder} - has image file",len(j))
        print(f"Folder: {folder} - has non-image file",len(i))

In [19]:
remove_non_image_file(my_data_dir='inputs/cherry-leaves_dataset/cherry-leaves')

Folder: healthy - has image file 2104
Folder: healthy - has non-image file 0
Folder: powdery_mildew - has image file 2104
Folder: powdery_mildew - has non-image file 0


### Split train validation set

In [20]:
import os
import shutil
import random
import joblib

my_data_dir="inputs/cherry-leaves_dataset/cherry-leaves"

def split_train_validation_test_images(my_data_dir, train_set_ratio, validation_set_ratio, test_set_ratio):
  
  if train_set_ratio + validation_set_ratio + test_set_ratio != 1.0:
    print("train_set_ratio + validation_set_ratio + test_set_ratio should sum 1.0")
    return

  # gets classes labels
  labels = os.listdir(my_data_dir) # it should get only the folder name
  if 'test' in labels:
    pass
  else: 
    # create train, test folders with classess labels sub-folder
    for folder in ['train','validation','test']:
      for label in labels:
        os.makedirs(name=my_data_dir+ '/' + folder + '/' + label)

    for label in labels:

      files = os.listdir(my_data_dir + '/' + label)
      random.shuffle(files)

      train_set_files_qty = int(len(files) * train_set_ratio)
      validation_set_files_qty = int(len(files) * validation_set_ratio)

      count = 1
      for file_name in files:
        if count <= train_set_files_qty:
          # move given file to train set
          shutil.move(my_data_dir + '/' + label + '/' + file_name,
                      my_data_dir + '/train/' + label + '/' + file_name)
          

        elif count <= (train_set_files_qty + validation_set_files_qty ):
          # move given file to validation set
          shutil.move(my_data_dir + '/' + label + '/' + file_name,
                      my_data_dir + '/validation/' + label + '/' + file_name)

        else:
          # move given file to test set
          shutil.move(my_data_dir + '/' + label + '/' + file_name,
                  my_data_dir + '/test/' +label + '/'+ file_name)
          
        count += 1

      os.rmdir(my_data_dir + '/' + label)

In [21]:
split_train_validation_test_images(my_data_dir=f"inputs/cherry-leaves_dataset/cherry-leaves",
                        train_set_ratio = 0.7,
                        validation_set_ratio=0.1,
                        test_set_ratio=0.2
                        )

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

## Push generated/new files from this Session to your GitHub repo

* .gitignore

In [22]:
!cat .gitignore

core.Microsoft*
core.mongo*
core.python*
env.py
__pycache__/
*.py[cod]
node_modules/
.github/
cloudinary_python.txt
kaggle.json

* Git status

In [23]:
!git status

_FREC_Pwd.M 4912.JPG
	inputs/cherry-leaves_dataset/cherry-leaves/validation/powdery_mildew/3d908213-1f46-40a1-b4d6-567d4f05edfe___FREC_Pwd.M 0287.JPG
	inputs/cherry-leaves_dataset/cherry-leaves/validation/powdery_mildew/3e888774-5b36-4346-8627-484de167ea88___FREC_Pwd.M 4982.JPG
	inputs/cherry-leaves_dataset/cherry-leaves/validation/powdery_mildew/40f6e95a-837c-4d31-a1b4-14351c25bd25___FREC_Pwd.M 4621.JPG
	inputs/cherry-leaves_dataset/cherry-leaves/validation/powdery_mildew/41493e9c-7e7b-49ce-b189-e5b772623a78___FREC_Pwd.M 4583.JPG
	inputs/cherry-leaves_dataset/cherry-leaves/validation/powdery_mildew/43289d9f-5de8-4b04-a455-24445a6efe54___FREC_Pwd.M 4750.JPG
	inputs/cherry-leaves_dataset/cherry-leaves/validation/powdery_mildew/441b415a-197a-4880-a373-6821f5a14725___FREC_Pwd.M 4880.JPG
	inputs/cherry-leaves_dataset/cherry-leaves/validation/powdery_mildew/441b415a-197a-4880-a373-6821f5a14725___FREC_Pwd.M 4880_flipLR.JPG
	inputs/cherry-leaves_dataset/cherry-leaves/validation/powdery_mildew

* Git add

In [24]:
!git add .

* Git commit

In [26]:
!git commit -am " Add data collection"

[main 6274060]  Add data collection
 1 file changed, 14 insertions(+), 4 deletions(-)


* Git Push

In [27]:
! git push

Enumerating objects: 3818, done.
Counting objects: 100% (3818/3818), done.
Delta compression using up to 4 threads
Compressing objects: 100% (3806/3806), done.
Writing objects: 100% (3809/3809), 48.79 MiB | 13.59 MiB/s, done.
Total 3809 (delta 6), reused 1 (delta 1), pack-reused 0
remote: Resolving deltas: 100% (6/6), completed with 3 local objects.
To https://github.com/HumaIlyas/mildew-detection-in-cherry-leaves
   249477d..6274060  main -> main


* If you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
    # create here your folder
    # os.makedirs(name='')
except Exception as e:
    print(e)
